In [3]:
##Dependencies
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
# Import label encoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pickle
#building a pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [4]:
##Loading data
df = (pd.read_feather('C:/Users/VIOLIN/Desktop/text-data-spr22/data/mtg.feather')[['name','text', 'mana_cost', 'flavor_text','release_date', 'edhrec_rank','color_identity']])
df

,name,text,mana_cost,flavor_text,release_date,edhrec_rank,color_identity
0,Ancestor's Chosen,First strike (This creature deals combat damag...,"[5, W, W]",<NA>,2007-07-13,16916.0,[W]
1,Angel of Mercy,Flying When Angel of Mercy enters the battlefi...,"[4, W]",Every tear shed is a drop of immortality.,2007-07-13,14430.0,[W]
2,Aven Cloudchaser,Flying (This creature can't be blocked except ...,"[3, W]",<NA>,2007-07-13,13098.0,[W]
3,Ballista Squad,"{X}{W}, {T}: Ballista Squad deals X damage to ...","[3, W]",The perfect antidote for a tightly packed form...,2007-07-13,14972.0,[W]
4,Bandage,Prevent the next 1 damage that would be dealt ...,[W],"Life is measured in inches. To a healer, every...",2007-07-13,4980.0,[W]
...,...,...,...,...,...,...,...
56361,Roiling Regrowth,Sacrifice a land. Search your library for up t...,"[2, G]","""Roots find their way through rock, and even m...",2020-09-25,736.0,[G]
56362,Kargan Warleader,Other Warriors you control get +1/+1.,"[1, R, W]",From rebellion against the Akoum Skyclave to f...,2020-09-25,8312.0,"[R, W]"
56363,"A-Omnath, Locus of Creation","When Omnath, Locus of Creation enters the batt...","[1, R, G, W, U]",<NA>,2020-09-25,NaN,"[G, R, U, W]"
56364,"A-Phylath, World Sculptor","Trample When Phylath, World Sculptor enters th...","[4, R, G]",<NA>,2020-09-25,NaN,"[G, R]"


In [5]:
df.shape #data exploration

(56366, 7)

In [6]:
df.dtypes#data exploration

name                      object
text                      string
mana_cost                 object
flavor_text               string
release_date      datetime64[ns]
edhrec_rank              float64
color_identity            object
dtype: object

In [7]:
df.isnull().any() #data exploration

name              False
text              False
mana_cost          True
flavor_text        True
release_date      False
edhrec_rank        True
color_identity    False
dtype: bool

In [8]:
df.isnull().sum()  #data exploration

name                  0
text                  0
mana_cost          7043
flavor_text       26731
release_date          0
edhrec_rank        4123
color_identity        0
dtype: int64

In [9]:
df = df.dropna() #data exploration

In [10]:
df.isnull().any() #data exploration

name              False
text              False
mana_cost         False
flavor_text       False
release_date      False
edhrec_rank       False
color_identity    False
dtype: bool

In [11]:
df.shape #data exploration

(27821, 7)

In [12]:
df['full_text'] = df['text'] + df['flavor_text'] ##preparing predictor variable

C:\Users\VIOLIN\AppData\Local\Temp/ipykernel_9060/935874284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['text'] + df['flavor_text'] ##preparing predictor variable


In [13]:
x = df['full_text'].fillna('') #handling nulls
x.isnull().any()

False

In [15]:
y = df['color_identity'] ##defining y for multiclass 
y = [list(i)[0] if len(i) == 1 else -1 for i in y] ##-1 for more than one color

In [16]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder() 
y = label_encoder.fit_transform(y)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y) ##splitting the data

In [19]:
##model pipeline
text_clf = Pipeline([
('vect', CountVectorizer(input='content',ngram_range=(1,1),max_df=25, min_df=5)), ##text preprocessing
('tfidf', TfidfTransformer()), ##tfidf
('clf', LinearSVC())]) #model

In [20]:
#fitting the data
text_clf.fit(x_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(max_df=25, min_df=5)),
                ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])

In [21]:
pickle.dump(text_clf, open('multiclass.sav', 'wb')) #saving the model as object